In [30]:
import pandas as pd

# Leer los datos directamente desde GitHub
url_airports = "https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat"
url_airlines = "https://raw.githubusercontent.com/jpatokal/openflights/master/data/airlines.dat"
url_routes = "https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat"

# Columnas para cada archivo (documentadas en OpenFlights)
columns_airports = [
    "Airport ID", "Name", "City", "Country", "IATA", "ICAO",
    "Latitude", "Longitude", "Altitude", "Timezone", "DST",
    "Tz database time zone", "Type", "Source"
]

columns_airlines = [
    "Airline ID", "Name", "Alias", "IATA", "ICAO", "Callsign",
    "Country", "Active"
]

columns_routes = [
    "Airline", "Airline ID", "Source airport", "Source airport ID",
    "Destination airport", "Destination airport ID",
    "Codeshare", "Stops", "Equipment"
]

# Cargar los DataFrames
df_airports = pd.read_csv(url_airports, header=None, names=columns_airports)
df_airlines = pd.read_csv(url_airlines, header=None, names=columns_airlines)
df_routes = pd.read_csv(url_routes, header=None, names=columns_routes)




In [18]:
df_airports.head()

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports


In [33]:
df_routes.head()

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment,Source Latitude,Destination Latitude,Source Latitude,Destination Latitude
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2,43.449902,55.606201,39.956600,49.278702
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2,46.283298,55.606201,48.006302,49.278702
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2,46.283298,44.225101,48.006302,43.081902
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2,55.305801,55.606201,61.503300,49.278702
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2,55.305801,55.012600,61.503300,82.650703


# Rutas 
Primero voy a modificar el dataframe que tenemos de las rutas para que podamos representarlo en un mapa en `Streamlit` usando la latidud y longitud de estos aeropuertos y graficarlo.

In [31]:
import itertools

combinaciones = list(itertools.product(["Latitude", "Longitude"], ["Source", "Destination"]))


In [32]:
for eje, posicion in combinaciones:
    df_routes = df_routes.merge(
        df_airports[["IATA", eje]],
        left_on=f"{posicion} airport",
        right_on="IATA",
        how="left"
    )

    df_routes.rename(columns={eje: f"{posicion} Latitude"}, inplace=True)
    df_routes.drop(columns=["IATA"], inplace=True)